In [1]:
import pandas as pd
import numpy as np
import pprint
import seaborn as sns
import matplotlib.pyplot as plt

In [56]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sub = pd.read_csv('./data/sample_submission.csv')
sec = pd.read_csv('./data/external_open/대구 보안등 정보_re.csv')
child = pd.read_csv('./data/external_open/대구 어린이 보호 구역 정보_re.csv')
park = pd.read_csv('./data/external_open/대구 주차장 정보_re.csv')
cctv = pd.read_csv('./data/external_open/대구 CCTV 정보_re.csv')

/var/folders/rc/3jzqjhp56_v2qxb49fkb1x880000gn/T/ipykernel_1573/4116356519.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  sec = pd.read_csv('./data/external_open/대구 보안등 정보_re.csv')


In [57]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [4]:
train #(39609, 23)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,승용,남,52세,경상,0,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,2021-12-31 19,금요일,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,승용,여,52세,상해없음,이륜,남,28세,경상,0,0,1,0,3
39605,ACCIDENT_39605,2021-12-31 19,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,측면충돌,안전거리미확보,승용,여,60세,상해없음,승용,남,52세,경상,0,0,1,0,3
39606,ACCIDENT_39606,2021-12-31 21,금요일,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차,측면충돌,교차로운행방법위반,승용,남,60세,중상,승용,남,73세,중상,0,2,0,0,10
39607,ACCIDENT_39607,2021-12-31 22,금요일,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,40세,상해없음,승용,여,57세,경상,0,0,1,0,3


#### test의 column들만 사용하여 train 데이터 구성

In [35]:
train = train[test.columns]

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      39609 non-null  object
 1   사고일시    39609 non-null  object
 2   요일      39609 non-null  object
 3   기상상태    39609 non-null  object
 4   시군구     39609 non-null  object
 5   도로형태    39609 non-null  object
 6   노면상태    39609 non-null  object
 7   사고유형    39609 non-null  object
dtypes: object(8)
memory usage: 2.4+ MB


### 도로명 주소를 지번 주소로 바꾼 후, 시군구 나눌 예정

'로'는 포함, '동'은 미포함

In [18]:
# # import sys
# # sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options
# import http.client

In [29]:
# wd = webdriver.Chrome('./chromedriver.exe')
# url = f"https://www.juso.go.kr/CommonPageLink.do?link=/support/AddressConvert"

AttributeError: 'str' object has no attribute 'capabilities'

In [ ]:
# #도로명 주소에서 지번 주소로 변경
# def change_adress(address):
#     wd.get(url)
#     scroll_down()
    

In [15]:
train[(train['시군구'].str.contains('로'))&(~train['시군구'].str.contains('동'))]

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
56,ACCIDENT_00056,2019-01-03 03,목요일,맑음,대구광역시 중구 남성로,교차로 - 교차로안,건조,차대차
92,ACCIDENT_00092,2019-01-04 05,금요일,맑음,대구광역시 중구 태평로3가,교차로 - 교차로횡단보도내,건조,차대사람
746,ACCIDENT_00746,2019-01-24 15,목요일,맑음,대구광역시 중구 태평로3가,단일로 - 기타,건조,차대차
870,ACCIDENT_00870,2019-01-27 22,일요일,맑음,대구광역시 중구 서문로2가,교차로 - 교차로부근,건조,차대차
1186,ACCIDENT_01186,2019-02-04 06,월요일,맑음,대구광역시 중구 태평로3가,교차로 - 교차로부근,건조,차대차
...,...,...,...,...,...,...,...,...
38251,ACCIDENT_38251,2021-11-20 20,토요일,맑음,대구광역시 중구 북성로2가,교차로 - 교차로안,건조,차대차
38473,ACCIDENT_38473,2021-11-26 18,금요일,맑음,대구광역시 중구 태평로3가,교차로 - 교차로안,건조,차대차
38663,ACCIDENT_38663,2021-12-02 12,목요일,맑음,대구광역시 중구 시장북로,교차로 - 교차로안,건조,차대차
38703,ACCIDENT_38703,2021-12-03 15,금요일,맑음,대구광역시 중구 태평로3가,단일로 - 기타,건조,차대차


### 도로명 주소는 삭제_selenium chrome driver 문제로 도로명주소 변환이 아닌 도로명주소 삭제

In [58]:
train = train[~((train['시군구'].str.contains('로')) & (~train['시군구'].str.contains('동')))]
train

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,승용,남,52세,경상,0,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,2021-12-31 19,금요일,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,승용,여,52세,상해없음,이륜,남,28세,경상,0,0,1,0,3
39605,ACCIDENT_39605,2021-12-31 19,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,측면충돌,안전거리미확보,승용,여,60세,상해없음,승용,남,52세,경상,0,0,1,0,3
39606,ACCIDENT_39606,2021-12-31 21,금요일,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차,측면충돌,교차로운행방법위반,승용,남,60세,중상,승용,남,73세,중상,0,2,0,0,10
39607,ACCIDENT_39607,2021-12-31 22,금요일,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,40세,상해없음,승용,여,57세,경상,0,0,1,0,3


In [59]:
traindf = train.copy()
testdf = test.copy()

In [60]:
location_pattern = r'(\S+) (\S+) (\S+)'

traindf[['도시', '구', '읍면동']] = train['시군구'].str.extract(location_pattern)
traindf = traindf.drop(columns=['시군구'])

testdf[['도시', '구', '읍면동']] = test['시군구'].str.extract(location_pattern)
testdf = testdf.drop(columns=['시군구'])

In [61]:
for i, item in enumerate(traindf['읍면동'].unique()):
    print(item, end = " ")
    if i%10 == 9:
        print("\n")

대신동 감삼동 두산동 복현동 신암동 지산동 상인동 태전동 지묘동 평리동 

신기동 상리동 월성동 황금동 구암동 신천동 만촌동 남산동 비산동 침산동 

두류동 신매동 대곡동 유가읍 논공읍 율하동 시지동 봉무동 다사읍 동산동 

이현동 이천동 수성동4가 노원동3가 서변동 관음동 학정동 파동 상동 이곡동 

효목동 읍내동 죽전동 대명동 동천동 지저동 사수동 덕산동 본리동 용계동 

범어동 신당동 성당동 용산동 도학동 욱수동 불로동 칠성동1가 장기동 매호동 

대천동 삼덕동1가 수성동2가 팔달동 삼덕동 진천동 수성동3가 가창면 방촌동 산격동 

월암동 현풍읍 구지면 매천동 송현동 고성동3가 봉덕동 칠성동2가 공평동 봉산동 

괴전동 국우동 대현동 노원동1가 대봉동 본동 신서동 검단동 화원읍 동호동 

삼덕동2가 입석동 중동 삼덕동3가 파호동 동내동 하빈면 율암동 동인동1가 내당동 

유천동 호산동 교동 옥포읍 원대동3가 검사동 동인동4가 수성동1가 고성동2가 각산동 

중리동 고성동1가 갈산동 대림동 호림동 연호동 동변동 범물동 계산동2가 동문동 

고모동 금호동 장동 도원동 수창동 동인동2가 가천동 노변동 달성동 동인동3가 

동성로2가 중대동 사월동 장관동 신평동 숙천동 성동 동일동 서호동 용수동 

대흥동 동성로3가 연경동 원대동1가 남일동 부동 송정동 백안동 도동 인교동 

미대동 노원동2가 조야동 포정동 노곡동 진인동 덕곡동 문화동 상매동 사복동 

계산동1가 상서동 원대동2가 미곡동 사일동 내동 전동 둔산동 매여동 완전동 

하서동 화전동 능성동 금강동 향촌동 수동 신무동 대안동 서야동 신용동 

동성로1가 평광동 도남동 

In [8]:
sec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71913 entries, 0 to 71912
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   보안등위치명    71913 non-null  object 
 1   설치개수      71913 non-null  int64  
 2   소재지도로명주소  34646 non-null  object 
 3   소재지지번주소   71913 non-null  object 
 4   위도        43602 non-null  float64
 5   경도        43602 non-null  float64
 6   설치연도      29397 non-null  float64
 7   설치형태      35373 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 4.4+ MB


# Sec

In [150]:
sec = pd.read_csv('./data/external_open/대구 보안등 정보_re.csv')

/var/folders/rc/3jzqjhp56_v2qxb49fkb1x880000gn/T/ipykernel_1573/3408689847.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  sec = pd.read_csv('./data/external_open/대구 보안등 정보_re.csv')


In [151]:
for i, item in enumerate(sec['소재지지번주소'].unique()):
    print(item, end = " ")
    if i%10 == 9:
        print("\n")

대구광역시 남구 대명동 1722-4 대구광역시 남구 대명동 1723-11 대구광역시 남구 대명동 1722-56 대구광역시 남구 대명동 1722-41 대구광역시 남구 대명동 1721-22 대구광역시 남구 대명동 1721-21 대구광역시 남구 대명동 1721-7 대구광역시 남구 대명동 1721-25 대구광역시 남구 대명동 1721-8 대구광역시 남구 대명동 1718-5 

대구광역시 남구 대명동 1220-2 대구광역시 남구 대명동 1220-4 대구광역시 남구 대명동 1716-1 대구광역시 남구 대명동 1716-2 대구광역시 남구 대명동 1716-5 대구광역시 남구 대명동 1717-5 대구광역시 남구 대명동 805-16 대구광역시 남구 대명동 1716-21 대구광역시 남구 대명동 1678-5 대구광역시 남구 대명동 1677-13 

대구광역시 남구 대명동 1671-6 대구광역시 남구 대명동 1715-25 대구광역시 남구 대명동 1716-12 대구광역시 남구 대명동 1715-16 대구광역시 남구 대명동 1670-5 대구광역시 남구 대명동 1715-13 대구광역시 남구 대명동 1669-12 대구광역시 남구 대명동 1663-16 대구광역시 남구 대명동 618-13 대구광역시 남구 대명동 619-20 

대구광역시 남구 대명동 619-4 대구광역시 남구 대명동 619-1 대구광역시 남구 대명동 803-6 대구광역시 남구 대명동 619-11 대구광역시 남구 대명동 1220-53 대구광역시 남구 대명동 619-3 대구광역시 남구 대명동 629-1 대구광역시 남구 대명동 1220-88 대구광역시 남구 대명동 1220-39 대구광역시 남구 대명동 1220-60 

대구광역시 남구 대명동 1220-11 대구광역시 남구 대명동 1220-80 대구광역시 남구 대명동 1677-25 대구광역시 남구 대명동 1672-12 대구광역시 남구 대명동 1677-19 대구광역시 남구 대명동 1677-1 대구광역시 남구 대명동 1677-15 대구광역시 남구 대명동 1671-7 대구광역시 남구 대명동 

### sec['소재지지번주소']에서 split("") 빈칸으로 나누고 세번째-> 읍면동 주소만 가져오기

In [152]:
sec['소재지지번주소'] = sec['소재지지번주소'].apply(lambda x: x.split(" "))
sec = sec[sec['소재지지번주소'].apply(len) >= 3]

filter_sec = sec.copy()
filter_sec['소재지지번주소'] = sec['소재지지번주소'].apply(lambda x: x[2])

In [153]:
filter_sec = filter_sec.drop(columns = ['보안등위치명', '소재지도로명주소', '설치연도'])

In [154]:
for i, item in enumerate(filter_sec['소재지지번주소'].unique()):
    print(item, end = " ")
    if i%10 == 9:
        print("\n")

대명동 봉덕동 이천동 감삼동 두류동 도원동 신당동 송현동 이곡동 갈산동 

용산동 이곡동1231-6 장기동 상인동 본동 본리동 성당동527-1 성당동526 성당동528 성당동529-5 

성당동520-14 성당동509 성당동516-2 성당동539-4 성당동530-1 성당동534-2 성당동534-1 성당동537-1 성당동538-2 성당동545 

장동 장기동157-1 장기동223-4 장기동270-12 성당동543 성당동542 성당동549-8 성당동640-3 성당동649-12 성당동642-1 

성당동649-7 성당동540 상인동1127-2 성당동648-1 성당동506-1 성당동647-5 성당동507-1 성당동647-1 성당동645-4 성당동645-10 

성당동643-5 성당동642-3 성당동641-8 성당동641-1 성당동645-1 성당동541-2 파호동 성당동489-2 성당동489-11 성당동 

성당동488-1 성당동487-22 성당동487-23 성당동487-35 성당동487-4 죽전동 월성동 월암동 대천동 성당동461-7 

성당동461-1 성당동461-3 성당동465-2 성당동465-14 성당동464-4 성당동464-1 성당동464-5 성당동465-6 성당동462-17 성당동463-12 

성당동463-10 성당동463-8 성당동462-20 월성동1788-11 성당동658-6 성당동658-13 성당동655-3 성당동656-9 성당동658-3 성당동654-7 

성당동654-10 성당동654-12 성당동654-3 성당동656-2 성당동657-1 성당동663-10 성당동726-1 성당동726-3 성당동666-1 성당동667-6 

성당동634-3 성당동637-7 성당동635-6 성당동635-1 성당동635-11 성당동636-12 성당동639-1 성당동638-6 성당동493-2 성당동493-7 

성당동492-6 성당동492-3 성당동491-11 성당동491-5 성당동490-1 성당동440 성당동441-1 성당동441-13 성당동441-9 성당동382-10 

In [155]:
#동 뒤에 붙은 숫자들 제거
filter_sec['소재지지번주소'] = filter_sec['소재지지번주소'].apply(lambda x: x.split('동')[0] + '동' if x.count('동') == 1 else x)

In [156]:
for i, item in enumerate(filter_sec['소재지지번주소'].unique()):
    print(item, end = " ")
    if i%10 == 9:
        print("\n")

대명동 봉덕동 이천동 감삼동 두류동 도원동 신당동 송현동 이곡동 갈산동 

용산동 장기동 상인동 본동 본리동 성당동 장동 파호동 죽전동 월성동 

월암동 대천동 진천동 대곡동 유천동 호산동 호림동 가창면 구지면 현풍면 

유가면 논공읍 옥포면 다사읍 화원읍 송촌리 유가읍 도의리 유곡리 한정리 

원교리 하빈면 능성동 진인동 백안동 신무동 미곡동 용수동 도학동 중대동 

신용동 덕곡동 송정동 미대동 내동 지묘동 신기동 도동 평광동 도평로131 

입석동 검사동 입석로 방촌동 불로동 봉무동 팔공로355-3 신암동 신천동 율하동 

용계동 율암동 상매동 매여동 각산동 서호동 동호동 신서동 괴전동 금강동 

대림동 숙천동 사복동 내곡동 동내동 지저동 신평동 부동 둔산동 해안동 

988-130 효목동 대현동 대현1동 대현2동 동변동 연경동 조야동 복현동 산격동 

산격1동 산격2동 산격3동 산격4동 검단동 노곡동 관음동 구암동 도남동 국우동 

학정동 침산1동 노원동 서변동 고성동 팔달동 복현1동 매천동 침산동 동천동 

신암5동 금호동 칠성동 태전2동 읍내동 침산3동 태전동 태전1동 평리4동 평리6동 

내당2·3동 평리2동 상중이동 내당1동 평리1동 내당4동 비산2·3동 평리3동 평리5동 비산7동 

비산1동 원대동 비산4동 파동 고산2동 만촌2동 수성2.3가동 황금2동 만촌3동 범어1동 

범어3동 지산1동 두산동 범어2동 범물1동 만촌1동 수성1가동 범어4동 고산3동 고산1동 

상동 수성4가동 황금1동 지산2동 범물2동 중동 동인동1가 동인동4가 동인동2가 동인동3가 

삼덕동 태평로1가 동 포정동 남일동 문화동 봉산동 남산동 동문동 공평동 

화전동 북성로1가 덕산동 용덕동 완전동 교동 향촌동 사일동 상덕동 동산동 

대안동 북성로2가 서문로1가 수창동 전동 수동 계산동 남성로 종로2가 장관동 

동일동 태평로2가 서내동 종로1가 상서동 태평로3가 북내동 서성로1가 서문로2가 서성로2가 

달성동 서야동 인교동 시장북로 대신동 대봉동 

In [157]:
# 동이름 사이에 들어간 숫자 제거
filter_sec['소재지지번주소'] = filter_sec['소재지지번주소'].apply(lambda x: "".join(char for char in x if not char.isdigit()) if x.endswith('동') & (len(x) <= 4) else x)

In [158]:
for i, item in enumerate(filter_sec['소재지지번주소'].unique()):
    print(item, end = " ")
    if i%10 == 9:
        print("\n")

대명동 봉덕동 이천동 감삼동 두류동 도원동 신당동 송현동 이곡동 갈산동 

용산동 장기동 상인동 본동 본리동 성당동 장동 파호동 죽전동 월성동 

월암동 대천동 진천동 대곡동 유천동 호산동 호림동 가창면 구지면 현풍면 

유가면 논공읍 옥포면 다사읍 화원읍 송촌리 유가읍 도의리 유곡리 한정리 

원교리 하빈면 능성동 진인동 백안동 신무동 미곡동 용수동 도학동 중대동 

신용동 덕곡동 송정동 미대동 내동 지묘동 신기동 도동 평광동 도평로131 

입석동 검사동 입석로 방촌동 불로동 봉무동 팔공로355-3 신암동 신천동 율하동 

용계동 율암동 상매동 매여동 각산동 서호동 동호동 신서동 괴전동 금강동 

대림동 숙천동 사복동 내곡동 동내동 지저동 신평동 부동 둔산동 해안동 

988-130 효목동 대현동 동변동 연경동 조야동 복현동 산격동 검단동 노곡동 

관음동 구암동 도남동 국우동 학정동 침산동 노원동 서변동 고성동 팔달동 

매천동 동천동 금호동 칠성동 태전동 읍내동 평리동 내당2·3동 상중이동 내당동 

비산2·3동 비산동 원대동 파동 고산동 만촌동 수성2.3가동 황금동 범어동 지산동 

두산동 범물동 수성1가동 상동 수성4가동 중동 동인동1가 동인동4가 동인동2가 동인동3가 

삼덕동 태평로1가 동 포정동 남일동 문화동 봉산동 남산동 동문동 공평동 

화전동 북성로1가 덕산동 용덕동 완전동 교동 향촌동 사일동 상덕동 동산동 

대안동 북성로2가 서문로1가 수창동 전동 수동 계산동 남성로 종로2가 장관동 

동일동 태평로2가 서내동 종로1가 상서동 태평로3가 북내동 서성로1가 서문로2가 서성로2가 

달성동 서야동 인교동 시장북로 대신동 대봉동 

In [159]:
# 도평로131, 팔공로355-3, 988-130 도로명주소라 삭제/ 내당2·3동, 수성2.3가동 구체적인 위치 파악 불가하므로 삭제
filter_sec = filter_sec[filter_sec['소재지지번주소']!='도평로131']
filter_sec = filter_sec[filter_sec['소재지지번주소']!='팔공로355-3']
filter_sec = filter_sec[filter_sec['소재지지번주소']!='988-130']
filter_sec = filter_sec[filter_sec['소재지지번주소']!='내당2·3동']
filter_sec = filter_sec[filter_sec['소재지지번주소']!='수성2.3가동']
filter_sec = filter_sec[filter_sec['소재지지번주소']!='비산2·3동']
filter_sec = filter_sec[filter_sec['소재지지번주소']!='동']

In [160]:
for i, item in enumerate(filter_sec['소재지지번주소'].unique()):
    print(item, end = " ")
    if i%10 == 9:
        print("\n")

대명동 봉덕동 이천동 감삼동 두류동 도원동 신당동 송현동 이곡동 갈산동 

용산동 장기동 상인동 본동 본리동 성당동 장동 파호동 죽전동 월성동 

월암동 대천동 진천동 대곡동 유천동 호산동 호림동 가창면 구지면 현풍면 

유가면 논공읍 옥포면 다사읍 화원읍 송촌리 유가읍 도의리 유곡리 한정리 

원교리 하빈면 능성동 진인동 백안동 신무동 미곡동 용수동 도학동 중대동 

신용동 덕곡동 송정동 미대동 내동 지묘동 신기동 도동 평광동 입석동 

검사동 입석로 방촌동 불로동 봉무동 신암동 신천동 율하동 용계동 율암동 

상매동 매여동 각산동 서호동 동호동 신서동 괴전동 금강동 대림동 숙천동 

사복동 내곡동 동내동 지저동 신평동 부동 둔산동 해안동 효목동 대현동 

동변동 연경동 조야동 복현동 산격동 검단동 노곡동 관음동 구암동 도남동 

국우동 학정동 침산동 노원동 서변동 고성동 팔달동 매천동 동천동 금호동 

칠성동 태전동 읍내동 평리동 상중이동 내당동 비산동 원대동 파동 고산동 

만촌동 황금동 범어동 지산동 두산동 범물동 수성1가동 상동 수성4가동 중동 

동인동1가 동인동4가 동인동2가 동인동3가 삼덕동 태평로1가 포정동 남일동 문화동 봉산동 

남산동 동문동 공평동 화전동 북성로1가 덕산동 용덕동 완전동 교동 향촌동 

사일동 상덕동 동산동 대안동 북성로2가 서문로1가 수창동 전동 수동 계산동 

남성로 종로2가 장관동 동일동 태평로2가 서내동 종로1가 상서동 태평로3가 북내동 

서성로1가 서문로2가 서성로2가 달성동 서야동 인교동 시장북로 대신동 대봉동 

In [161]:
#도로명으로 끝나는 데이터 삭제
filter_sec = filter_sec[~filter_sec['소재지지번주소'].str.endswith('로')]

In [162]:
for i, item in enumerate(filter_sec['소재지지번주소'].unique()):
    print(item, end = " ")
    if i%10 == 9:
        print("\n")

대명동 봉덕동 이천동 감삼동 두류동 도원동 신당동 송현동 이곡동 갈산동 

용산동 장기동 상인동 본동 본리동 성당동 장동 파호동 죽전동 월성동 

월암동 대천동 진천동 대곡동 유천동 호산동 호림동 가창면 구지면 현풍면 

유가면 논공읍 옥포면 다사읍 화원읍 송촌리 유가읍 도의리 유곡리 한정리 

원교리 하빈면 능성동 진인동 백안동 신무동 미곡동 용수동 도학동 중대동 

신용동 덕곡동 송정동 미대동 내동 지묘동 신기동 도동 평광동 입석동 

검사동 방촌동 불로동 봉무동 신암동 신천동 율하동 용계동 율암동 상매동 

매여동 각산동 서호동 동호동 신서동 괴전동 금강동 대림동 숙천동 사복동 

내곡동 동내동 지저동 신평동 부동 둔산동 해안동 효목동 대현동 동변동 

연경동 조야동 복현동 산격동 검단동 노곡동 관음동 구암동 도남동 국우동 

학정동 침산동 노원동 서변동 고성동 팔달동 매천동 동천동 금호동 칠성동 

태전동 읍내동 평리동 상중이동 내당동 비산동 원대동 파동 고산동 만촌동 

황금동 범어동 지산동 두산동 범물동 수성1가동 상동 수성4가동 중동 동인동1가 

동인동4가 동인동2가 동인동3가 삼덕동 태평로1가 포정동 남일동 문화동 봉산동 남산동 

동문동 공평동 화전동 북성로1가 덕산동 용덕동 완전동 교동 향촌동 사일동 

상덕동 동산동 대안동 북성로2가 서문로1가 수창동 전동 수동 계산동 종로2가 

장관동 동일동 태평로2가 서내동 종로1가 상서동 태평로3가 북내동 서성로1가 서문로2가 

서성로2가 달성동 서야동 인교동 대신동 대봉동 

In [165]:
filter_sec

,설치개수,소재지지번주소,위도,경도,설치형태
0,1,대명동,35.846703,128.579803,한전주
1,1,대명동,35.846863,128.579406,한전주
2,1,대명동,35.846341,128.579342,건축물
3,2,대명동,35.846368,128.578922,한전주
4,1,대명동,35.845995,128.578858,한전주
...,...,...,...,...,...
71908,1,대봉동,35.857556,128.600023,한전주
71909,1,대봉동,35.859205,128.600046,한전주
71910,1,대봉동,35.859442,128.599627,한전주
71911,1,대봉동,35.858244,128.597614,건축물


In [163]:
result = filter_sec.groupby(['소재지지번주소', '설치형태'])['설치개수'].sum()
pd.DataFrame(result)

설치개수
소재지지번주소 설치형태      
각산동     건축물      1
        전용주      9
        한전주    129
검사동     건축물     48
        전용주     10
...            ...
황금동     전용주     49
        한전주    637
효목동     건축물    304
        전용주     21
        한전주    839

[278 rows x 1 columns]

In [166]:
result['대명동']['한전주']

3704

In [169]:
result = filter_sec[filter_sec['설치형태'] == '한전주'].groupby('소재지지번주소')['설치개수'].sum().reset_index()

# '한전주형태의보안등개수' 컬럼을 추가하고, 해당 소재지지번주소에 대응하는 '한전주' 설치형태의 총 개수를 할당
filter_sec['한전주형태의보안등개수'] = filter_sec['소재지지번주소'].map(result.set_index('소재지지번주소')['설치개수'])
filter_sec

,설치개수,소재지지번주소,위도,경도,설치형태,한전주형태의보안등개수
0,1,대명동,35.846703,128.579803,한전주,3704.0
1,1,대명동,35.846863,128.579406,한전주,3704.0
2,1,대명동,35.846341,128.579342,건축물,3704.0
3,2,대명동,35.846368,128.578922,한전주,3704.0
4,1,대명동,35.845995,128.578858,한전주,3704.0
...,...,...,...,...,...,...
71908,1,대봉동,35.857556,128.600023,한전주,321.0
71909,1,대봉동,35.859205,128.600046,한전주,321.0
71910,1,대봉동,35.859442,128.599627,한전주,321.0
71911,1,대봉동,35.858244,128.597614,건축물,321.0


In [170]:
result = filter_sec[filter_sec['설치형태'] == '건축물'].groupby('소재지지번주소')['설치개수'].sum().reset_index()

# '한전주형태의보안등개수' 컬럼을 추가하고, 해당 소재지지번주소에 대응하는 '한전주' 설치형태의 총 개수를 할당
filter_sec['건축물보안등개수'] = filter_sec['소재지지번주소'].map(result.set_index('소재지지번주소')['설치개수'])
result = filter_sec[filter_sec['설치형태'] == '전용주'].groupby('소재지지번주소')['설치개수'].sum().reset_index()

# '한전주형태의보안등개수' 컬럼을 추가하고, 해당 소재지지번주소에 대응하는 '한전주' 설치형태의 총 개수를 할당
filter_sec['전용주보안등개수'] = filter_sec['소재지지번주소'].map(result.set_index('소재지지번주소')['설치개수'])
filter_sec

,설치개수,소재지지번주소,위도,경도,설치형태,한전주형태의보안등개수,건축물보안등개수,전용주보안등개수
0,1,대명동,35.846703,128.579803,한전주,3704.0,1649.0,22.0
1,1,대명동,35.846863,128.579406,한전주,3704.0,1649.0,22.0
2,1,대명동,35.846341,128.579342,건축물,3704.0,1649.0,22.0
3,2,대명동,35.846368,128.578922,한전주,3704.0,1649.0,22.0
4,1,대명동,35.845995,128.578858,한전주,3704.0,1649.0,22.0
...,...,...,...,...,...,...,...,...
71908,1,대봉동,35.857556,128.600023,한전주,321.0,199.0,26.0
71909,1,대봉동,35.859205,128.600046,한전주,321.0,199.0,26.0
71910,1,대봉동,35.859442,128.599627,한전주,321.0,199.0,26.0
71911,1,대봉동,35.858244,128.597614,건축물,321.0,199.0,26.0


In [49]:
sec['소재지지번주소'][0].split(" ")[2]

'대명동'